In [1]:
import pandas as pd
import numpy as np
import os
import io
import gzip


1. Lectura de los archivos en raw, y me creo los archivos parquet que voy a utilizar

In [2]:
df = pd.read_csv('../../data/raw/train.csv')
df_test_nolabel = pd.read_csv('../../data/raw/test_nolabel.csv')
df_sample_submission= pd.read_csv('../../data/raw/sample_submission.csv')

# df_train = df_test_nolabel
#df_train = pd.read_parquet('../data/agregado_diario_2019-2021.parquet')

In [3]:
df

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
0,bd9d6267ec5,1523195006,"P-SCAPE LAND DESIGN, LLC",NORTHFIELD,OH,CITIZENS BANK NATL ASSOC,RI,1-Nov-05,2006,2,...,0,2,0,1,N,N,31-Dec-05,"$8,000.00",$0.00,1
1,9eebf6d8098,1326365010,The Fresh & Healthy Catering C,CANTON,OH,"FIRSTMERIT BANK, N.A.",OH,6-Jun-05,2005,2,...,1,2,1,1,N,N,31-Jul-05,"$166,000.00",$0.00,1
2,83806858500,6179584001,AARON MASON & HOWE LLC,SAWYERWOOD,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,18-Mar-03,2003,2,...,4,2,1,2,Y,N,31-Mar-03,"$25,000.00",$0.00,1
3,a21ab9cb3af,8463493009,MID OHIO CAR WASH,COLUMBUS,OH,THE HUNTINGTON NATIONAL BANK,OH,28-Jun-95,1995,2,...,0,0,1,0,N,N,31-Jan-96,"$220,100.00",$0.00,1
4,883b5e5385e,3382225007,Bake N Brew LLC,Newark,OH,THE HUNTINGTON NATIONAL BANK,OH,16-Apr-09,2009,0,...,0,0,0,1,N,N,31-May-09,"$25,000.00",$0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22830,4f9443d2a46,1573725008,"SIBILA RACE ENGINEERING, INC",MASSILLON,OH,CITIZENS BANK NATL ASSOC,RI,9-Dec-05,2006,1,...,0,1,0,1,N,N,31-Dec-05,"$70,000.00",$0.00,1
22831,798db2753a7,2011184008,ENVIRO SHIELD POWER WASHING,SPRINGBORO,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,27-Apr-98,1998,2,...,0,0,1,0,N,Y,31-May-98,"$30,000.00",$0.00,1
22832,ddb3c5e9bff,4082983001,"MAINLINE TRCK&TRAILR SRVC, INC",BEDFORD,OH,GROWTH CAPITAL CORP.,OH,9-May-90,1990,16,...,6,10,1,0,N,N,13-Feb-91,"$92,000.00",$0.00,1
22833,407200a5dfe,7783283010,TIN BOX STUDIO,CINCINNATI,OH,KEYBANK NATIONAL ASSOCIATION,OH,10-Nov-94,1995,1,...,0,0,1,0,N,Y,31-Jan-95,"$20,000.00",$0.00,1


2. Cambiar los valores correspondientes.

La gran mayoría de los valores son de tipo objet, int64 y float64, tienen que ser acorde a sus valores.

Primero cambio los valores de las fechas, que son Approval Date y DisbursementDate que están puesto con letra escrita en vez de en el formato de numero.
Después cambiamos los valores que están raros y no se adaptan

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22835 entries, 0 to 22834
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 22835 non-null  object 
 1   LoanNr_ChkDgt      22835 non-null  int64  
 2   Name               22834 non-null  object 
 3   City               22834 non-null  object 
 4   State              22835 non-null  object 
 5   Bank               22813 non-null  object 
 6   BankState          22813 non-null  object 
 7   ApprovalDate       22835 non-null  object 
 8   ApprovalFY         22835 non-null  int64  
 9   NoEmp              22835 non-null  int64  
 10  NewExist           22833 non-null  float64
 11  CreateJob          22835 non-null  int64  
 12  RetainedJob        22835 non-null  int64  
 13  FranchiseCode      22835 non-null  int64  
 14  UrbanRural         22835 non-null  int64  
 15  RevLineCr          22744 non-null  object 
 16  LowDoc             227

* *DisbursementDate* => transformar la fecha
* *Name* => String, Si no sabemos lo ponemos Unknown
* *City* => String, Si no sabemos lo ponemos Unknown
* *State* => Son todosde tipo OH
* *Bank* => Si quitamos los None es de tipo String, Si no sabemos lo ponemos Unknown
* *BankState* => Si quitamos los None es de tipo String, Si no sabemos lo ponemos Unknown
* *NewExist* => 2 / 1 / 0, int, y poner la media donde sea nulo; 

    1 = Existing business, 2 = New business; 

    Si no lo sabemos ponemos vieja empresa que es la mayoría, es decir, que ya existía la empresa, 0. 

    Por lo que corresponde con 0 = Existing y 1 = New
* *RevLineCr* => array(['N', 'Y', '0', 'T', None, '`'], dtype=object, 

    Hay que limpiarlo, sabiendo que tiene que ser Y = Yes, N = No, para el que no sepamos ponemos la mayoría; después ponemos True => 1 y False => 0
* *LowDoc* => array(['N', 'Y', None, 'A', '0', 'S', 'R', 'C'], dtype=object)

    Tiene que ser Y = Yes, N = No, para el que no sepamos ponemos la mayoría; después ponemos True => 1 y False => 0
* *DisbursementGross* => Quitar el dolar y hacerlo double
* *BalanceGross* => Quitar el dolar y hacerlo double y donde sea nulo

In [5]:
import pandas as pd
import numpy as np

def conversion(df):
    """
    Función para convertir el DataFrame
    
    Args:
    - df: DataFrame con los datos
    
    Returns:
    - df_train
    """
    # Copiar el DataFrame
    df_train = df.copy()
    df_train = df_train.drop(columns=["id","LoanNr_ChkDgt","Name","State","Bank"])
    ### Conversion de valores 
    # Conversión de fechas a formato Datetime
    df_train['ApprovalDate'] = pd.to_datetime(df_train['ApprovalDate'])
    df_train['DisbursementDate'] = pd.to_datetime(df_train['DisbursementDate'])

    ### Limpiar y convertir a float 'DisbursementGross' y 'BalanceGross'
    df_train['DisbursementGross'] = df_train['DisbursementGross'].replace('[\$,]', '', regex=True).astype(float)
    df_train['BalanceGross'] = df_train['BalanceGross'].replace('[\$,]', '', regex=True).astype(float)
    return df_train

# Conversion de datos
df_train = conversion(df)
df_train = df_train.drop(columns=["id","LoanNr_ChkDgt","Name","State"])
df_train.info()

df_train.isnull().sum()

df_train_accept_0 = df_train[df_train["Accept"]==0]
df_train_accept_0.isnull().sum()

In [6]:
def binarizarCategoria(df_train,column,column_grouped):

    # Sustituyo nulos y extraños por la moda
    df_train[column].fillna(df_train[column].mode()[0], inplace=True)
    # Cambio a enteros
    df_train[column] = df_train[column].astype("int64")
    
    df_train[column_grouped] = 0
    df_train.loc[
        df_train[column] > 0,
        column_grouped
    ] = int(1)
    
    # Elimino la columna que ya no sirve
    df_train.drop(columns=column, inplace=True)
    return df_train

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def agrupar_en_rangos(df, columna, target='Accept', bins=[-1, 40000, 87500, 200000, np.inf], include_lowest=True, labels=['Muy bajo', 'Bajo', 'Medio', 'Alto']):
    """
    Agrupa una columna numérica en rangos definidos, crea dummies y grafica la relación con la variable objetivo.
    
    Parámetros:
    - df: DataFrame original
    - columna: nombre de la columna numérica a agrupar
    - target: columna objetivo (por defecto 'Accept')
    - bins: lista de cortes para los rangos
    - labels: etiquetas para los rangos
    
    Retorna:
    - df con la nueva columna de rangos y dummies concatenadas
    """
    
    # Verificar que la columna sea numérica
    if not np.issubdtype(df[columna].dtype, np.number):
        print(f"La columna '{columna}' no es numérica.")
        return df
    print(df.isnull().sum())

    # Crear columna de rangos
    rango_col = f"{columna}_rango"
    df[rango_col] = pd.cut(df[columna], bins=bins, labels=labels)
    
    # Crear dummies
    #dummies = pd.get_dummies(df[rango_col], prefix=columna)
    #df = pd.concat([df, dummies], axis=1)

    if 'Accetp' in df.columns:
        # Tasa de aceptación por rango
        if target in df.columns:
            tasa = df.groupby(rango_col)[target].mean().reset_index()
            plt.figure(figsize=(8, 5))
            sns.barplot(data=tasa, x=rango_col, y=target)
            plt.title(f'Tasa de {target} por rango de {columna}')
            plt.ylabel(f'Tasa de {target}')
            plt.xlabel(f'Rango de {columna}')
            plt.tight_layout()
            plt.show()
        else:
            print(f"⚠️ La variable objetivo '{target}' no está en el DataFrame.")
    
    return df

#df = agrupar_en_rangos(df, 'BalanceGross')



In [8]:
def formatear(df):

    ### City    
    # Transformar a mayúsculas y eliminar texto entre paréntesis. Los nulos (1) se sustituyen por la moda.
    df_train = df.copy()
    df_train['City'].fillna(df_train['City'].mode()[0], inplace=True) # se rellenan con la moda
    df_train["City"] = df_train["City"].str.upper()
    df_train = df_train[~df_train["City"].str.contains(r"\d", regex=True, na=False)]
    df_train["City"] = df_train["City"].str.replace(r" \(.*", "", regex=True).str.strip()

    ### Bank. Los nulos se pueden eliminar ya que no están en la clase minoritaria y la moda puede generar ruido
    #df_train['Bank'].fillna('Unknown', inplace=True)
    #df_train.dropna(subset=["Bank"])
    #df_train["Bank"] = df_train["Bank"].str.upper()

    ### BankState - Se comprueba que todos son dos letras. OH es muy mayoritario, se puede sustituir los valores por la moda
    df_train['BankState'].fillna(df_train['BankState'].mode()[0], inplace=True)

    ### Limpiar la columna 'ApprovalFY' para extraer solo los dígitos
    df_train['ApprovalFY'] = df_train['ApprovalFY'].astype(str).str.extract('(\d+)', expand=False)
    df_train['ApprovalFY'] = pd.to_numeric(df_train['ApprovalFY'], errors='coerce')

    ### NewExist - un valor nulo con Accept = 0 - se elimina
    df_train = df_train.dropna(subset=['NewExist'])
    df_train['NewExist'].astype('int64')
    df_train['NewExist'] = df_train['NewExist'].apply(
        lambda x: 1 if x == 2 else (0 if x == 1 else -1)
    )
    df_train = df_train[df_train['NewExist'] != -1]

    ### Crear la columna 'Franchise' basada en 'FranchiseCode'
    if 'FranchiseCode' in df_train.columns:
        df_train['Franchise'] = df_train['FranchiseCode'].apply(
            lambda x: 0 if x in [0, 1] else 1
        )
        df_train.drop(columns=['FranchiseCode'], inplace=True)

    ### RevLineCr - solo hay un Accept = 0 en las null, podemos rellenar con la moda
    if df_train['RevLineCr'].nunique() != 2:
        df_train['RevLineCr'] = df_train['RevLineCr'].apply(lambda x: 1 if str(x).strip().upper() in ['Y','T'] 
                                                                else (0 if str(x).strip().upper() in ['N','0'] 
                                                                      else np.nan))
        df_train['RevLineCr'] = df_train['RevLineCr'].astype("Int64")
        mode = df_train['RevLineCr'].mode()
        df_train['RevLineCr'].fillna(mode[0], inplace=True)

    ### LowDoc - Hay 24 valores con Accept = 0 - rellenamos con la moda
    df_train['LowDoc'] = df_train['LowDoc'].apply(lambda x: 1 if str(x).strip().upper() in ['Y'] 
                                                      else (0 if str(x).strip().upper() in ['N','0'] 
                                                            else np.nan))
    df_train['LowDoc'] = df_train['LowDoc'].astype("Int64")
    mode = df_train['LowDoc'].mode()
    df_train['LowDoc'].fillna(mode[0], inplace=True)

    ### DisbursementDate 
    # 1. Restar DisbursementDate de ApprovalDate para obtener la diferencia
    df_train['DisbursementInterval'] = (df_train['DisbursementDate'] - df_train['ApprovalDate']).dt.days
    
    # 2. Calcular la desviación estándar de la diferencia
    std_dev = df_train['DisbursementInterval'].std()
    
    # 3. Rellenar los valores nulos de DisbursementDate con ApprovalDate + desviación estándar
    df_train['DisbursementDate'] = df_train['DisbursementDate'].fillna(df_train['ApprovalDate'] + pd.Timedelta(days=std_dev))
    df_train.drop(columns=['DisbursementDate',"DisbursementInterval","ApprovalDate"], inplace=True)

    ### CreateJob - binarizar
    df_train = binarizarCategoria(df_train,"CreateJob","CreateJobBinary")
    ### RetainedJob - binarizar
    df_train = binarizarCategoria(df_train,"RetainedJob","RetainedJobBinary")

    ### Agrupacion
    df_train = agrupar_en_rangos(df_train, 'DisbursementGross')
    df_train = agrupar_en_rangos(df_train, 'BalanceGross', bins = [-1,0,10000, float('inf')],labels=["Sin Saldo","Saldo bajo","Saldo Alto"])
    return df_train



# Conversion de datos
print(df_train['CreateJobBinary'].value_counts(normalize=True) * 100)
df_train['RetainedJobBinary'].value_counts(normalize=True) * 100


import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

# Funcion para ver la distribucion de una variable
def showDistribution(category,percentil):
    # Crear un boxplot para visualizar la distribución de 'CreateJob' en función de 'accept'
    filtered_df = df_train.copy()
    filtered_df_2 = df_train.copy()
    # Configurar el tamaño de la figura y subplots
    fig, axes = plt.subplots(2, 2, figsize=(14, 6)) 
    
    # Primer boxplot: CreateJob vs accept
    sns.boxplot(x='Accept', y=category, data=df_train, ax=axes[0,0])
    axes[0,0].set_title('CreateJob según accept')
    
    percentiles = [0.25, 0.5, 0.75, 0.9, 0.95, 0.99, percentil]
    print(df_train.shape[0])
    
    percents = df_train[category].quantile(percentil)
    
    # Filtrar los datos que son menores o iguales al percentil 97.5
    filtered_df = df_train[df_train[category] <= percents]
    filtered_df.describe()
    
    # Segundo boxplot: por ejemplo, CreateJob vs otra variable categórica
    sns.boxplot(x='Accept', y='CreateJob', data=filtered_df, ax=axes[0,1])
    axes[0,1].set_title('CreateJob según Accept filtrado')

    # Tercer boxplot: RetainedJob vs Accept
    sns.boxplot(x='Accept', y='RetainedJob', data=df_train, ax=axes[1,0])
    axes[1,0].set_title('CreateJob según accept')
    columna = "CreateJob"

    filtered_df_2 = df_train[df_train["RetainedJob"] <= percents]
    filtered_df.describe()
    
    # Cuarto boxplot: por ejemplo, CreateJob vs otra variable categórica
    sns.boxplot(x='Accept', y='RetainedJob', data=filtered_df_2, ax=axes[1,1])
    axes[1,1].set_title('CreateJob según Accept filtrado')
    
    # Ajustar espaciado
    plt.tight_layout()
    plt.show()

    group0 = df[df['Accept'] == 0]['CreateJob']
    group1 = df[df['Accept'] == 1]['CreateJob']
    ttest_ind(group0, group1)

    print("Filas eliminadas por CreateJob:", df_train.shape[0] - filtered_df.shape[0])
    print("Filas eliminadas por RetainedJob:", df_train.shape[0] - filtered_df_2.shape[0])

    print(filtered_df[category].describe())
    print(filtered_df_2["RetainedJob"].describe())
showDistribution("DisbursementGross", 0.998)


In [9]:
df = pd.read_csv('../../data/raw/train.csv')
df_test_nolabel = pd.read_csv('../../data/raw/test_nolabel.csv')
df_sample_submission= pd.read_csv('../../data/raw/sample_submission.csv')

In [10]:
# Para guardar el archivo como df_train.csv

# Conversion de datos
df_train = conversion(df)

# Formateo
df_train_fitted = formatear(df_train)

output_path = "../../data/processed/train_processed_v3.csv"
df_train_fitted.to_csv(output_path, index=False)
print("Archivo guardado en:", output_path)

/tmp/ipykernel_120279/283548590.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_train['ApprovalDate'] = pd.to_datetime(df_train['ApprovalDate'])
/tmp/ipykernel_120279/283548590.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_train['DisbursementDate'] = pd.to_datetime(df_train['DisbursementDate'])


City                 0
BankState            0
ApprovalFY           0
NoEmp                0
NewExist             0
UrbanRural           0
RevLineCr            0
LowDoc               0
DisbursementGross    0
BalanceGross         0
Accept               0
Franchise            0
CreateJobBinary      0
RetainedJobBinary    0
dtype: int64
City                       0
BankState                  0
ApprovalFY                 0
NoEmp                      0
NewExist                   0
UrbanRural                 0
RevLineCr                  0
LowDoc                     0
DisbursementGross          0
BalanceGross               0
Accept                     0
Franchise                  0
CreateJobBinary            0
RetainedJobBinary          0
DisbursementGross_rango    0
dtype: int64
Archivo guardado en: ../../data/processed/train_processed_v3.csv


In [11]:
# Conversion de datos
df_train = conversion(df_test_nolabel)

# Formateo
df_train_fitted = formatear(df_train)

output_path = "../../data/processed/test_nosample_processed.csv"
df_train_fitted.to_csv(output_path, index=False)
print("Archivo guardado en:", output_path)

/tmp/ipykernel_120279/283548590.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_train['ApprovalDate'] = pd.to_datetime(df_train['ApprovalDate'])


City                 0
BankState            0
ApprovalFY           0
NoEmp                0
NewExist             0
UrbanRural           0
RevLineCr            0
LowDoc               0
DisbursementGross    0
BalanceGross         0
Franchise            0
CreateJobBinary      0
RetainedJobBinary    0
dtype: int64
City                       0
BankState                  0
ApprovalFY                 0
NoEmp                      0
NewExist                   0
UrbanRural                 0
RevLineCr                  0
LowDoc                     0
DisbursementGross          0
BalanceGross               0
Franchise                  0
CreateJobBinary            0
RetainedJobBinary          0
DisbursementGross_rango    0
dtype: int64
Archivo guardado en: ../../data/processed/test_nosample_processed.csv


/tmp/ipykernel_120279/283548590.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_train['DisbursementDate'] = pd.to_datetime(df_train['DisbursementDate'])


df_train = agrupar_en_rangos(df_train, 'DisbursementGross')
df_train = agrupar_en_rangos(df_train, 'BalanceGross', bins = [-1,0,10000, float('inf')],labels=["Sin Saldo","Saldo bajo","Saldo Alto"])
df_train.head()